In [105]:
import pandas as pd
import requests
import io
import altair as alt
import functools


In [17]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
r = requests.get(url)
data = r.content.decode('utf8')

In [110]:
url_states = "https://covidtracking.com/api/v1/states/daily.json"
r_states = requests.get(url_states)
states_data = r_states.content.decode('utf8')
states_df = pd.read_json(states_data)

@functools.lru_cache
def fetch_data():
    url_states = "https://covidtracking.com/api/v1/states/daily.json"
    return requests.get(url_states).content.decode('utf8')

In [115]:
class CovidApi:
    def __init__(self):
        self.underlying = pd.read_json(fetch_data())
        self.underlying["date"] = self.underlying["date"].astype("str").astype("datetime64")
    
    @property
    def states(self):
        return set(self.underlying["state"])
    
    def filter_by(self, state = None, date = None):
        """
        Arg:
            date: formate yyyymmdd
        """
        data = self.underlying
        assert not state or state in self.states, f"state must be on of {self.states}"
        if state:
            data = data[data["state"] == state]
        if date:
            data = data[data["date"] == date]
        return data
    
    def plot_by(self, state, prop):
        if state:
            data = self.filter_by(state=state)
        else:
            data = self.underlying.groupby("date").sum().reset_index()
        return alt.Chart(data).mark_bar().encode(
            x='date',
            y=prop
        )

In [116]:
covid_api = CovidApi()

In [123]:
covid_api.plot_by(state="FL", prop="death")

alt.Chart(...)